In [11]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from pdbfixer import PDBFixer
import numpy as np

In [12]:
def setup_simulation_with_free_calcium(
    pdb_file,
    ca_concentration_mM=10,
    simulation_time_ns=200,
    use_drude=False
):
    """
    Complete workflow with free Ca2+ ions
    """
    
    # 1. Prepare system
    print("Preparing system...")
    fixer = PDBFixer(filename=pdb_file)
    fixer.findMissingResidues()
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(7.0)
    
    # 2. Add solvent
    print("Adding solvent...")
    fixer.addSolvent(
        padding=1.5*nanometers,
        ionicStrength=0.15*molar
    )
    
    modeller = Modeller(fixer.topology, fixer.positions)
    
    # 3. Calculate and add Ca2+ ions
    num_ca = calculate_num_ions_from_concentration(
        modeller, 
        ca_concentration_mM
    )
    
    print(f"Adding {num_ca} free Ca2+ ions...")
    modeller = replace_waters_with_calcium(modeller, num_ca)
    
    # 4. Setup force field
    if use_drude:
        forcefield = ForceField('drude2013.xml')
        system = forcefield.createSystem(
            modeller.topology,
            nonbondedMethod=PME,
            nonbondedCutoff=1.2*nanometers,
            constraints=HBonds,
            polarization='drude'
        )
        integrator = DrudeLangevinIntegrator(
            300*kelvin, 1/picosecond,
            1*kelvin, 20/picosecond,
            0.001*picoseconds
        )
    else:
        forcefield = ForceField(
            'charmm36.xml',
            'charmm36/water.xml'
        )
        system = forcefield.createSystem(
            modeller.topology,
            nonbondedMethod=PME,
            nonbondedCutoff=1.2*nanometers,
            constraints=HBonds
        )
        integrator = LangevinMiddleIntegrator(
            300*kelvin,
            1/picosecond,
            0.002*picoseconds
        )
    
    # 5. Add barostat
    system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))
    
    # 6. Create simulation
    platform = Platform.getPlatformByName('CUDA')
    simulation = Simulation(
        modeller.topology,
        system,
        integrator,
        platform
    )
    simulation.context.setPositions(modeller.positions)
    
    # 7. Minimize and equilibrate
    print("Minimizing...")
    simulation.minimizeEnergy()
    
    print("Equilibrating...")
    simulation.context.setVelocitiesToTemperature(300*kelvin)
    simulation.step(50000)  # 100 ps
    
    # 8. Production run
    print(f"Production run ({simulation_time_ns} ns)...")
    simulation.reporters.append(
        DCDReporter('trajectory.dcd', 5000)
    )
    simulation.reporters.append(
        StateDataReporter(
            'log.txt', 5000,
            step=True,
            potentialEnergy=True,
            temperature=True,
            progress=True,
            remainingTime=True,
            totalSteps=simulation_time_ns*500000
        )
    )
    # Calculate steps based on actual timestep
    if use_drude:
        steps_per_ns = 1000000  # 1 fs timestep → 1,000,000 steps/ns
    else:
        steps_per_ns = 500000   # 2 fs timestep → 500,000 steps/ns

    # Production run
    simulation.step(int(simulation_time_ns * steps_per_ns))
    simulation.step(int(simulation_time_ns * 500000))
    
    print("Simulation complete!")
    return simulation


In [ ]:
simulation = setup_simulation_with_free_calcium(
    'protein.pdb',
    ca_concentration_mM=10,
    simulation_time_ns=50,
    use_drude=True  # Use Drude for best accuracy
)